In [ ]:
#Import Library
import pandas as pd

In [ ]:
#Import Dataset
data=pd.read_csv('database/data/external/Police_Use_of_Force.csv')

In [ ]:
#show data columns
data.columns

In [ ]:
#add year, month, day, hour into dataset
data['date']=data['ResponseDate'].apply(lambda x:x.split(' ')[0])
data['time']=data['ResponseDate'].apply(lambda x:x.split(' ')[1])
data['year']=data['date'].apply(lambda x:x.split('/')[0])
data['month']=data['date'].apply(lambda x:x.split('/')[1])
data['day']=data['date'].apply(lambda x:x.split('/')[2])
data['hour']=data['time'].apply(lambda x:x.split(':')[0])

In [ ]:
#drop duplicates columns
data = data.drop(['X','Y','CenterX','CenterY','DateAdded'],axis=1)

In [ ]:
#columns 
data.columns

In [ ]:
data.head()

# subject_tb

In [ ]:
subject_tb = data[['OBJECTID','Race','Sex','EventAge','SubjectInjury','SubjectRole','SubjectRoleNumber','TypeOfResistance']]
subject_tb = subject_tb.rename(columns={'OBJECTID':'subject_id','Race':'race','Sex':'sex','EventAge':'age',
                                     'SubjectInjury':'has_injury','SubjectRole':'role',
                                     'SubjectRoleNumber':'role_number','TypeOfResistance':'resistance'})
subject_tb['age'].fillna(0,inplace=True)
subject_tb['role_number'].fillna(0,inplace=True)
subject_tb['resistance'].fillna("Null",inplace=True)
subject_tb['sex'].fillna('Null',inplace=True)
subject_tb['sex'].replace('not recorded','Null',inplace=True)
subject_tb['role']=subject_tb['role'].apply(lambda x: str(x).strip().replace('PERSON IN CRISIS','PIC'))
subject_tb['race'].fillna("Null",inplace=True)

subject_tb['age'] = subject_tb['age'].astype('int')
subject_tb['role_number'] = subject_tb['role_number'].astype('int')
subject_tb.head()

In [ ]:
subject_tb.to_csv('database/data/external/subject_tb.csv',index=False)

# city_tb

In [ ]:
city_tb = pd.DataFrame({'city_id':0,'city_name':'Minneapolis'},index=[0])
city_tb.head()

In [ ]:
city_tb.to_csv('database/data/external/city_tb.csv',index=False)

# city_summary_tb

In [ ]:
city_summary_tb = data[['year','TotalCityCallsForYear']]
city_summary_tb = city_summary_tb.drop_duplicates()
city_summary_tb = city_summary_tb.reset_index(drop=True)
city_summary_tb = city_summary_tb.reset_index()
city_summary_tb['city_id'] = 0
city_summary_tb = city_summary_tb[['index','city_id','year','TotalCityCallsForYear']]
city_summary_tb = city_summary_tb.rename(columns={'index':'city_summary_id','TotalCityCallsForYear':'total_calls'})
city_summary_tb.head()

In [ ]:
city_summary_tb.to_csv('database/data/external/city_summary_tb.csv',index=False)

# precinct_tb

In [ ]:
precinct_list=list(data['Precinct'].unique())
precinct_list[-2]='Null'
precinct_dict={}
n=0
for i in (precinct_list):
    n=n+1
    precinct_dict.update({i:n})

In [ ]:
precinct_tb = data['Precinct'].drop_duplicates().fillna("Null").reset_index(drop=True).reset_index()
precinct_tb['index'] = precinct_tb['Precinct'].apply(lambda x:precinct_dict[x])
precinct_tb = precinct_tb.rename(columns={'index':'precinct_id','Precinct':"precinct_name"})
precinct_tb.head()

In [ ]:
precinct_tb.to_csv('database/data/external/precinct_tb.csv',index=False)

# precinct_summary

In [ ]:
precinct_raw = data['Precinct'].fillna("Null").reset_index(drop=True).reset_index()
precinct_raw['index'] = precinct_raw['Precinct'].apply(lambda x:precinct_dict[x])
precinct_raw =precinct_raw.rename(columns={'index':'precinct_id','Precinct':"precinct_name"})

In [ ]:
precinct_summary_tb_b4=precinct_raw.join(data['year'])
precinct_summary_tb_b4['count']=1

In [ ]:
total=(precinct_summary_tb_b4.groupby(['precinct_id','precinct_name','year']).sum())['count']
total_list = list(total)

In [ ]:
precinct_summary_tb = precinct_summary_tb_b4.sort_values(['precinct_id','precinct_name','year']).drop_duplicates().reset_index(drop=True).reset_index()
precinct_summary_tb = precinct_summary_tb.rename(columns = {'index':'precinct_summary_id'})
precinct_summary_tb = precinct_summary_tb.drop(['count','precinct_name'],axis=1)
precinct_summary_tb['total_calls'] = total_list
precinct_summary_tb.head()

In [ ]:
precinct_summary_tb.to_csv('database/data/external/precinct_summary_tb.csv',index=False)

# neightborhood_tb

In [ ]:
neighborhood_list = list(data['Neighborhood'].fillna('Null').drop_duplicates())
neighborhood_dict={}
n=0
for i in neighborhood_list:
    n=n+1
    neighborhood_dict.update({i:n})

In [ ]:
neighborhood_tb = pd.DataFrame({'neighborhood_id':list(neighborhood_dict.values()),
                                'neighborhood_name':list(neighborhood_dict.keys())})

In [ ]:
neighborhood_tb.head()

In [ ]:
neighborhood_tb.to_csv('database/data/external/neighborhood_tb.csv',index=False)

# neightborhood_summary_tb

In [ ]:
neighborhood_summary_raw = data[['Neighborhood','year']].copy()
neighborhood_summary_raw['count']=1
neighborhood_summary_raw['Neighborhood'].fillna('Null')
neighborhood_summary_raw['neighborhood_id']=neighborhood_summary_raw['Neighborhood'].fillna('Null')\
                                            .apply(lambda x:neighborhood_dict[x])
neighborhood_summary_raw = neighborhood_summary_raw.drop('Neighborhood',axis=1)

In [ ]:
total=(neighborhood_summary_raw.groupby(['neighborhood_id','year']).sum())['count']
total_list = list(total)

In [ ]:
neighborhood_summary_tb = neighborhood_summary_raw.copy()
neighborhood_summary_tb = neighborhood_summary_tb.drop_duplicates().sort_values(['neighborhood_id','year'])
neighborhood_summary_tb = neighborhood_summary_tb[['neighborhood_id','year']]
neighborhood_summary_tb['total_calls'] = total_list
neighborhood_summary_tb = neighborhood_summary_tb.reset_index(drop=True).reset_index()
neighborhood_summary_tb = neighborhood_summary_tb.rename(columns={'index':'neigborhood_summary_id'})
neighborhood_summary_tb.head()

In [ ]:
neighborhood_summary_tb.to_csv('database/data/external/neighborhood_summary_tb.csv',index=False)

# force_categories_tb

In [ ]:
force_categories_tb = data['ForceType'].copy()
force_categories_tb.fillna("Null",inplace=True)
force_categories_tb = force_categories_tb.drop_duplicates().reset_index(drop=True).reset_index()
force_categories_tb = force_categories_tb.rename(columns={'index':'force_category_id','ForceType':'category'})
force_categories_tb.head()

In [ ]:
force_categories_tb.to_csv('database/data/external/force_categories_tb.csv',index=False)

# police_force

In [ ]:
force_dict={}
for i in range(len(force_categories_tb)):
    force_dict.update({force_categories_tb.iloc[i,1]:force_categories_tb.iloc[i,0]})

In [ ]:
force_dict

In [ ]:
data.columns

In [ ]:
police_force_tb = data[['ForceType','OBJECTID','PoliceUseOfForceID','ForceReportNumber','ForceTypeAction']].copy()
police_force_tb.fillna("Null",inplace=True)
police_force_tb['force_category_id'] = police_force_tb['ForceType'].apply(lambda x: force_dict[x])
police_force_tb = police_force_tb.reset_index(drop=True).reset_index()
police_force_tb = police_force_tb[["index",'PoliceUseOfForceID','force_category_id',
                                   'ForceTypeAction','ForceReportNumber','OBJECTID']]
police_force_tb = police_force_tb.rename(columns={'index':'police_force_id','PoliceUseOfForceID':'force_number',
                                                 'ForceTypeAction':'force_action','ForceReportNumber':"force_report_number",
                                                 'OBJECTID':'subject_id'})
police_force_tb.head()

In [ ]:
police_force_tb.to_csv('database/data/external/police_force_tb_tb.csv',index=False)

# case_tb

In [ ]:
import datetime

In [ ]:
case_tb=data[['OBJECTID','CaseNumber','Is911Call','Problem','PrimaryOffense','ResponseDate','CenterLatitude',
             'CenterLongitude','Precinct','Neighborhood']]
case_tb=case_tb.rename(columns={'OBJECTID':'case_id','CaseNumber':'case_number','Is911Call':'is_911_call',
                       'Problem':'problem','PrimaryOffense':'primary_offense','CenterLatitude':'latitude',
                       'CenterLongitude':'longitude','Precinct':'precinct_id','Neighborhood':'neighborhood_id'})

case_tb['date']=case_tb['ResponseDate'].apply(lambda x:x.split(' ')[0])
case_tb['time']=case_tb['ResponseDate'].apply(lambda x:x.split(' ')[1])
case_tb['year']=case_tb['date'].apply(lambda x:x.split('/')[0])
case_tb['month']=case_tb['date'].apply(lambda x:x.split('/')[1])
case_tb['day']=case_tb['date'].apply(lambda x:x.split('/')[2])
case_tb['hour']=case_tb['time'].apply(lambda x:int(x.split(':')[0]))
case_tb['date']=case_tb['date'].apply(lambda x:x.replace("/",""))


case_tb['neighborhood_id'] = case_tb['neighborhood_id'].fillna('Null')
case_tb['neighborhood_id']=case_tb['neighborhood_id'].apply(lambda x:neighborhood_dict[x])
case_tb['city_id']=0
case_tb=case_tb.drop(['ResponseDate','time'],axis=1)
case_tb['hour']=case_tb['hour'].fillna("-1")
case_tb['police_force_id']=case_tb['case_id'].apply(lambda x: x-1)
case_tb['problem'].fillna('Null',inplace=True)
case_tb['precinct_id'].fillna('Null',inplace=True)
case_tb['precinct_id']=case_tb['precinct_id'].apply(lambda x: precinct_dict[x])
case_tb['primary_offense'].fillna("Null",inplace=True)


case_tb=case_tb[['case_id','case_number','is_911_call','problem','primary_offense','date','latitude',
                'longitude','city_id','precinct_id','neighborhood_id','police_force_id','year','month','day','hour']]
case_tb.head()

In [ ]:
case_tb.to_csv('database/data/external/case_tb.csv',index=False)